In [5]:
!pip install transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.3 MB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import transformers
import torch

# Original Claire-7B

In [11]:

model_name = "OpenLLM-France/Claire-7B-0.1"

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model_base = transformers.AutoModelForCausalLM.from_pretrained(model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    load_in_4bit=True                          # For efficient inference, if supported by the GPU card
)

pipeline_base = transformers.pipeline("text-generation", model=model_base, tokenizer=tokenizer)
generation_kwargs = dict(
    num_return_sequences=1,                    # Number of variants to generate.
    return_full_text= False,                   # Do not include the prompt in the generated text.
    max_new_tokens=200,                        # Maximum length for the output text.
    do_sample=True, top_k=10, temperature=1.0, # Sampling parameters.
    pad_token_id=tokenizer.eos_token_id,       # Just to avoid a harmless warning.
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:

prompt = """- Donne moi la recette pour faire un bon mojito\
-"""
completions = pipeline_base(prompt, **generation_kwargs)
for completion in completions:
    print(prompt + " […]" + completion['generated_text'])



- Donne moi la recette pour faire un bon mojito- […]trempe-salé
- C'est une recette de famille
- Ah
- Tu peux la faire en trente secondes Mais je te préviens c'est une recette à risque
- Je ne risque rien
- Il y a de l'alcool
- Tu peux faire avec du jus d'orange
- Je vais pas faire ça
- Non non
- Mais il faut que tu fasses quelque chose
- Tu peux faire avec des fruits
- Tu peux faire avec du citron
- Tu peux faire avec du citron
- C'est une recette de famille
- Ah bon C'est une recette de famille Mais il y a de l'alcool C'était quoi déjà le truc
- Non il faut que tu fasses quelque chose C'est une recette de famille
- C'est une recette de famille
- Mais je te préviens il y a de l'alcool


# Claire-7b-Instruct

In [5]:
tokenizer = transformers.AutoTokenizer.from_pretrained("OpenLLM-France/Claire-7B-0.1")

In [8]:
model_name = "PhilSad/Claire-7b-0.1-instruct"

tokenizer = transformers.AutoTokenizer.from_pretrained("OpenLLM-France/Claire-7B-0.1")
model_instruct = transformers.AutoModelForCausalLM.from_pretrained(model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

In [9]:
pipeline_instruct = transformers.pipeline("text-generation", model=model_instruct, tokenizer=tokenizer)
generation_kwargs = dict(
    num_return_sequences=1,                    # Number of variants to generate.
    return_full_text= False,                   # Do not include the prompt in the generated text.
    max_new_tokens=200,                        # Maximum length for the output text.
    do_sample=True, top_k=10, temperature=1.0, # Sampling parameters.
    pad_token_id=tokenizer.eos_token_id,       # Just to avoid a harmless warning.
)

In [10]:

prompt = """Ci-dessous se trouve une instruction qui décrit une tâche. Écrivez une réponse qui complète de manière appropriée la demande.

### Instruction :
Donne moi la recette pour faire un bon mojito

### Réponse :"""
completions = pipeline_instruct(prompt, **generation_kwargs)
for completion in completions:
    print(prompt + " […]" + completion['generated_text'])



/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Ci-dessous se trouve une instruction qui décrit une tâche. Écrivez une réponse qui complète de manière appropriée la demande.

### Instruction :
Donne moi la recette pour faire un bon mojito

### Réponse : […]
Le mojito est un cocktail alcoolisé originaire des Antilles, et c'est maintenant l'un des cocktails les plus appréciés. Les ingrédients du mojito comprennent du citron vert, du sirop de sucre, de l'eau gazeuse, de la menthe fraîche et de l'épeautre. La première étape pour faire un bon cocktail Mojito est de couper le citron vert en fines rondelles avec un couteau. Vous devez ensuite frotter le bord de votre verre avec le citron vert, et le mettre avec vos autres ingrédients. Vous devez ensuite verser de l'eau gazeuse dans le verre, et le faire glisser. Pour finir, vous devez mettre les feuilles de menthe fraîche et les bâtonnets de sucre dans le verre, et servir la boisson à vos invités! Bonne chance et bonne dégust
